In [1]:
from openalea.mtg import MTG

|Code varname|Invenio varname|
|---|---|
|Experiment_name	STRING||										
|Sample_date	DD/MM/YYYY|Date prelevement|								
|Architecture_date	DD/MM/YYYY||													
|Genotype	STRING |Lot|										
|Modality	STRING||											
|Plant_ID	INT|plant|											
|Stade	STRING|Stade HT pour l'inflorence, stade 19 à I pour les buds|
|Foliar_type	STRING # Cotyledon, Unifoliate, Trifoliate, Trifoliate non developped, primordia foliar	
|DBI	REAL # mm Collar diameter| Diamètre|
|INFLOLG	REAL # mm Inflorescence length| Hauteur HT|
|LFTLG_RIGHT	REAL # mm Right leaflet length|LG|
|LFTLG_CENTRAL	REAL # mm Central leaflet length|LS|
|LFTLG_LEFT	REAL # mm Left leaflet length||
|LFTWD_RIGHT	REAL # mm Right leaflet width||
|LFTWD_CENTRAL	REAL # mm Central leaflet width||
|LFTWD_LEFT	REAL # mm Left leaflet width||
|LFTAR_RIGHT	REAL # mm2 Right Leaflet area||
|LFTAR_CENTRAL	REAL # mm2 Central Leaflet area||
|LFTAR_LEFT	REAL # mm2 Left Leaflet area||
|FLWRNUMBER	INT # Total flower number||
|FLWRNUMBER_OPEN	INT # Total opened Flower number||
|FLWRNUMBER_ABORTED	INT # Total aborted Flower number||
|FLWRNUMBER_CLOSED	INT # Total closed flower number||


In [2]:

def main_crown(
    experimental_name="",
    sample_date="2014/12/10",
    genotype="Ciflorette",
    plant_id=1,
    nb_old_leaf=2,
    nb_develop_leaf=7,
    nb_reduce_leaf=2,
    nb_hide_leaf=2,
    Petiol_lenght=10.5,
    LS=9.3,
    LG=7.1,
    Diameter=27.1, 
    stage= "89",
    nb_flowers=0,
    nb_open_flower=0):
    
    g = MTG()
    plant_id = g.add_component(g.root, 
                               label='P',
                               Experimental_name=experimental_name,
                               Genotype=genotype,
                               Plant_ID=plant_id,
                               date=sample_date)
    trunk= g.add_component(plant_id, label="T",DBI=Diameter)

    nb_leaf= nb_old_leaf + nb_develop_leaf -1
    nb_hide_leaf=nb_reduce_leaf + nb_hide_leaf
    
    leaf=g.add_component(trunk,label="F")

    for i in range(nb_leaf-1):
        leaf=g.add_child(leaf,label="F",edge_type="<")
    
    leaf=g.add_child(leaf,label="F",edge_type="<",LFTLG_RIGHT=LG,LFTLG_CENTRAL=LS,PETLG=Petiol_lenght)
        
    for i in range(nb_leaf,nb_leaf+nb_hide_leaf):
        leaf=g.add_child(leaf,label="f",edge_type="<")
    
    if  stage in ("17","18","19"):
        inflo = g.add_child(leaf,label="bt",edge_type="<",Stade=stage)
    elif  stage in ("A","B","C","D","E","F","G","H"):
        inflo = g.add_child(leaf,label="ht",edge_type="<",Stade=stage)
    else:
        inflo = g.add_child(leaf,label="HT",edge_type="<",Stade=stage,)

    return g

In [3]:
def branch_crown(g,
                 vid,
                 nb_leaf=2,
                nb_hide_leaf=3,
                stage="17"):
    vid,branch=g.add_child_and_complex(vid,label="F",edge_type="+")
    g.node(branch).label='A'
    g.node(branch).edge_type='+'
    
    for i in range(nb_leaf-1):
        vid=g.add_child(vid,label='F', edge_type='<')
    
    for i in range (nb_hide_leaf):
        vid=g.add_child(vid,label='f', edge_type='<')
    
    if  stage in ("17","18","19"):
        inflo = g.add_child(vid,label="bt",edge_type="<",Stade=stage)
    elif  stage in ("A","B","C","D","E","F","G","H"):
        inflo = g.add_child(vid,label="ht",edge_type="<",Stade=stage)
    else:
        inflo = g.add_child(vid,label="HT",edge_type="<",Stade=stage)
        
    return g


In [4]:
def bud(g,vids,stage='17'):

    for vid in vids:
        vid,bud=g.add_child_and_complex(vid,label="bt",edge_type="+",Stade=stage)
        g.node(bud).label='A'
        g.node(bud).edge_type='+'

    return g
    


In [5]:
def stolon(g,vids):

    for vid in vids:
        vid,stolon=g.add_child_and_complex(vid,label="s",edge_type="+")
        g.node(stolon).label='A'
        g.node(stolon).edge_type='+'

    return g
    


In [6]:
g=main_crown()
bud(g,vids=[4,5,6])
branch_crown(g,8)
stolon(g,[15])

In [7]:
g.property_names()


['edge_type',
 'label',
 'Experimental_name',
 'Genotype',
 'Plant_ID',
 'date',
 'DBI',
 'LFTLG_RIGHT',
 'LFTLG_CENTRAL',
 'PETLG',
 'Stade']

In [71]:
def get_leaf_base(g):
    return [v for v in g.vertices() if g.label(v)=='F' and g.parent(v)==None]

In [72]:
get_leaf_base(g)

[3]

In [73]:
[v for v in g.vertices() if g.label(v)=='F' and g.parent(v)==get_leaf_base(g)[0]]

[4]

# Test pour Marie

In [15]:
g=main_crown(nb_develop_leaf=8,nb_reduce_leaf=0,nb_hide_leaf=0,nb_old_leaf=0)
bud(g,vids=[3])
branch_crown(g,vid=4,nb_leaf=1,nb_hide_leaf=0)
bud(g,vids=[15])
branch_crown(g,vid=5,nb_leaf=2, nb_hide_leaf=0)
bud(g,vids=[20,21])
branch_crown(g,vid=6,nb_leaf=2, nb_hide_leaf=0,stage='I')
bud(g,vids=[28])
branch_crown(g,vid=29,nb_leaf=2, nb_hide_leaf=0,stage='I')
bud(g,vids=[34])
#branch_crown(g,vid=29,nb_leaf=2, nb_hide_leaf=0,stage='I')

In [16]:
g.display()

MTG : nb_vertices=39, nb_scales=4
/P			(id=1)											
^/T			(id=2)											
^/F			(id=3)											
	+A			(id=12)										
	^/bt			(id=13)										
^<F			(id=4)											
	+A			(id=14)										
	^/F			(id=15)										
		+A			(id=17)									
		^/bt			(id=18)									
	^<bt			(id=16)										
^<F			(id=5)											
	+A			(id=19)										
	^/F			(id=20)										
		+A			(id=23)									
		^/bt			(id=24)									
	^<F			(id=21)										
		+A			(id=25)									
		^/bt			(id=26)									
	^<bt			(id=22)										
^<F			(id=6)											
	+A			(id=27)										
	^/F			(id=28)										
		+A			(id=31)									
		^/bt			(id=32)									
	^<F			(id=29)										
		+A			(id=33)									
		^/F			(id=34)									
			+A			(id=37)								
			^/bt			(id=38)								
		^<F			(id=35)									
		^<HT			(id=36)									
	^<HT			(id=30)										
^<F			(id=7)											
^<F			(id=8)											
^<F			(id=9)											
^<F			(id=10)											
^<HT			(id=11)											


In [17]:
import openalea.core

ModuleNotFoundError: No module named 'openalea.core'

In [18]:
g.get_vertex_property(1)

{'label': 'P',
 'Experimental_name': '',
 'Genotype': 'Ciflorette',
 'Plant_ID': 1,
 'date': '2014/12/10'}

In [26]:
vertices=g.vertices()
g.add_property("index") 


TypeError: properties() takes 1 positional argument but 2 were given

In [27]:
g.property("index")= g.vertice

{}